In [ ]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image
from torchvision.utils import make_grid
import PIL

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import glob
import random

from kaggle_datasets import KaggleDatasets

import tensorflow as tf

In [ ]:
GCS_PATH = '/kaggle/input/gan-getting-started/' #KaggleDatasets().get_gcs_path()

In [ ]:
class Downsample(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1, apply_instancenorm=True):
        super(Downsample, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=nn.InstanceNorm2d)
        self.norm = nn.InstanceNorm2d(out_channels)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.apply_norm = apply_instancenorm

    def forward(self, x):
        x = self.conv(x)
        if self.apply_norm:
            x = self.norm(x)
        x = self.relu(x)

        return x

In [ ]:
class Upsample(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1, dropout=True):
        super(Upsample, self).__init__()
        self.dropout = dropout
        self.block = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=nn.InstanceNorm2d),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        self.dropout_layer = nn.Dropout2d(0.5)

    def forward(self, x, shortcut=None):
        x = self.block(x)
        if self.dropout:
            x = self.dropout_layer(x)

        if shortcut is not None:
            x = torch.cat([x, shortcut], dim=1)

        return x

In [ ]:
class CycleGAN_Unet_Generator(nn.Module):
    def __init__(self, filter=64):
        super(CycleGAN_Unet_Generator, self).__init__()
        self.downsamples = nn.ModuleList([
            Downsample(3, filter, kernel_size=4, apply_instancenorm=False),  # (b, filter, 128, 128)
            Downsample(filter, filter * 2),  # (b, filter * 2, 64, 64)
            Downsample(filter * 2, filter * 4),  # (b, filter * 4, 32, 32)
            Downsample(filter * 4, filter * 8),  # (b, filter * 8, 16, 16)
            Downsample(filter * 8, filter * 8), # (b, filter * 8, 8, 8)
        ])

        self.upsamples = nn.ModuleList([
            Upsample(filter * 8, filter * 8),
            Upsample(filter * 16, filter * 4, dropout=False),
            Upsample(filter * 8, filter * 2, dropout=False),
            Upsample(filter * 4, filter, dropout=False)
        ])

        self.last = nn.Sequential(
            nn.ConvTranspose2d(filter * 2, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        skips = []
        for l in self.downsamples:
            x = l(x)
            skips.append(x)

        skips = reversed(skips[:-1])
        for l, s in zip(self.upsamples, skips):
            x = l(x, s)

        out = self.last(x)

        return out

In [ ]:
#train(False)
!conda install -y gdown


In [ ]:
# download the saved model
!gdown https://drive.google.com/uc?id=1gXK17UPrG7gVnjfTJKbBNwLlZAh0vc9p

In [ ]:
#MODEL_PATH = "/kaggle/working/cycleGAN_5700.pth"
import torchvision.models as models

gen_PM = CycleGAN_Unet_Generator()
gen_PM.load_state_dict(torch.load('cycleGAN_pytorchN.pth'))


In [ ]:
#to delete
print(os.getcwd())
if os.path.isdir('../images') == False:
  os.makedirs('../images')


In [ ]:
class ImageTransform:
    def __init__(self, img_size=256):
        self.transform = {
            'train': transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5])
            ]),
            'test': transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5], std=[0.5])
            ])}

    def __call__(self, img, phase='train'):
        img = self.transform[phase](img)

        return img


# Monet Dataset ---------------------------------------------------------------------------
class MonetDataset(Dataset):
    def __init__(self, base_img_paths, style_img_paths,  transform, phase='train'):
        self.base_img_paths = base_img_paths
        self.style_img_paths = style_img_paths
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return min([len(self.base_img_paths), len(self.style_img_paths)])

    def __getitem__(self, idx):        
        base_img_path = self.base_img_paths[idx]
        style_img_path = self.style_img_paths[idx]
        base_img = Image.open(base_img_path)
        style_img = Image.open(style_img_path)

        base_img = self.transform(base_img, self.phase)
        style_img = self.transform(style_img, self.phase)

        return base_img, style_img

In [ ]:
transform = ImageTransform(img_size=256)

In [ ]:
import shutil
def submit(net, transform):
    os.makedirs('../images', exist_ok=True)
    
    net.eval()
    photo_img_paths = tf.io.gfile.glob(str(GCS_PATH + '/photo_jpg/*.*'))
    
    for path in photo_img_paths:
        photo_id = path.split('/')[-1]
        img = transform(Image.open(path), phase='test')
        
        gen_img = net(img.unsqueeze(0))[0]
        
        # Reverse Normalization
        gen_img = gen_img * 0.5 + 0.5
        gen_img = gen_img * 255
        gen_img = gen_img.detach().cpu().numpy().astype(np.uint8)
        
        gen_img = np.transpose(gen_img, [1,2,0])
        
        gen_img = Image.fromarray(gen_img)
        gen_img.save(os.path.join('../images', photo_id))
        
    # Make Zipfile
    shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")
    
    # Delete Origin file
    shutil.rmtree('../images')

In [ ]:
submit(gen_PM, transform)